In [1]:
import os
os.chdir("/afs/csail.mit.edu/u/s/samsl/Work/Adapting_PLM_DTI")

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.metrics import average_precision_score
from sklearn.model_selection import KFold
from collections import defaultdict
from collections.abc import Iterable
from tqdm.notebook import tqdm

In [3]:
from src.featurizers import (
    ProtBertFeaturizer,
    MorganFeaturizer,
)
from src.data import (
    get_task_dir,
    DTIDataModule,
)
from src.utils import (
    set_random_seed,
    get_logger,
)

In [9]:
def flatten(xs):
    for x in xs:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            yield from flatten(x)
        else:
            yield x

In [4]:
device = torch.device("cuda:0")
seed = 61998
set_random_seed(seed)
logg = get_logger()

In [73]:
dset_list = ['biosnap', 'biosnap_prot', 'bindingdb', 'davis']

In [95]:
def summarize_replicates(df):
    drug_list = [i[0] for i in df.keys()]
    rep_list = [i[1] for i in df.keys()]
    scores = df.values()
    df = pd.DataFrame({'Task':drug_list,'Rep': rep_list, 'AUPR': scores})
    return df.groupby('Task').mean()

def display_results(meta, tal, tap, tad):
    dset, drug_uniq, train_only, skipped, total = meta
    
    print(f"Data Set {dset}:")
    print(f"Total drugs in training set: {len(drug_uniq)}")
    print(f"Train only tasks: {train_only}")
    print(f"Number of test tasks skipped: {skipped}")
    print(f"Pct of test tasks skipped: {(skipped / total):.2%}")
    
    task_auprs = list(tad.values())
    avg_of_avg = np.nanmean(task_auprs)
    print(f"Average per-task AUPR: {avg_of_avg}")
    
    replicates = list(set([i[1] for i in tal.keys()]))
    
    rep_auprs = []
    for r in replicates:
        tal_rep = {k:v for (k,v) in tal.items() if k[1] == r}
        tap_rep = {k:v for (k,v) in tap.items() if k[1] == r}
        labels = list(flatten(list(tal_rep.values())))
        preds = list(flatten(list(tap_rep.values())))
        rep_auprs.append(average_precision_score(labels, preds))
    print(f"Overall AUPR: {np.mean(rep_auprs)} +- {np.std(rep_auprs)}")

def benchmark_ridge(dset, n_replicates):
    
    # Set Up Data
    task_dir = get_task_dir(dset)
    drug_featurizer = MorganFeaturizer(save_dir=task_dir)
    target_featurizer = ProtBertFeaturizer(save_dir=task_dir)
    
    datamodule = DTIDataModule(
        task_dir,
        drug_featurizer,
        target_featurizer,
        device=device,
    )
    datamodule.setup()
    
    drug_column = datamodule._drug_column
    target_column = datamodule._target_column
    label_column = datamodule._label_column
    
    # Load Embeddings
    train_df = datamodule.df_train
    test_df = datamodule.df_test
    full_df = pd.concat([datamodule.df_train, datamodule.df_test])

    drug_uniq = full_df[drug_column].unique()
    target_uniq = full_df[target_column].unique()
    drug_featurizer.preload(drug_uniq)
    target_featurizer.preload(target_uniq)
    
    # Initialize tracking
    dset_sizes = {}

    all_predictions = defaultdict(list)
    all_cpi_predictions = defaultdict(list)
    all_labels = defaultdict(list)
    task_aupr_dict = {}
    skipped = 0
    train_only = 0

    # For each drug
    for curr_task in tqdm(drug_uniq,total=len(drug_uniq)):

        # Generate featurizers and train/test subsets
        drug_feat = drug_featurizer(curr_task)
        
        train_df_task = train_df[train_df[drug_column] == curr_task]
        test_df_task = test_df[test_df[drug_column] == curr_task]

        dset_sizes[curr_task] = (len(train_df_task), len(test_df_task))
        if (len(train_df_task) < 1) or (len(test_df_task) < 1):
            if curr_task in test_df[drug_column].unique():
                skipped += 1
            else:
                train_only += 1
            continue
            
        train_X = []
        for i, r in train_df_task.iterrows():
            train_X.append(target_featurizer(r[target_column]))
        train_X = torch.stack(train_X, 0).detach().cpu().numpy()
        train_Y = train_df_task[label_column].values
        assert len(train_X) == len(train_Y)
        
        test_X = []
        for i, r in test_df_task.iterrows():
            test_X.append(target_featurizer(r[target_column]))
        test_X = torch.stack(test_X, 0).detach().cpu().numpy()
        test_Y = test_df_task[label_column].values
        assert len(test_X) == len(test_Y)

        # For each replicate
        for r in range(n_replicates):
            
            # Fit a model
            model = Ridge(random_state=r)
            model.fit(train_X, train_Y)
            
            # Make and store predictions
            prd = model.predict(test_X)
            curr_aupr = average_precision_score(test_Y, prd)
            task_aupr_dict[(curr_task,r)] = curr_aupr 

            all_labels[(curr_task, r)].append(test_Y)
            all_predictions[(curr_task, r)].append(prd)
    
    total = len(test_df[drug_column].unique())
    meta = dset, drug_uniq, train_only, skipped, total
        
    return meta, all_labels, all_predictions, task_aupr_dict

In [96]:
results = {}

for dset in dset_list:
    meta, tal, tap, tad = benchmark_ridge(dset,5)
    results[dset] = (meta, tal, tap, tad)

Morgan:  54%|██████████████████████████████████████████████████████████████████████████▊                                                               | 2443/4510 [00:00<00:00, 3018.59it/s][15:05:27] Unusual charge on atom 0 number of radical electrons set to zero
[15:05:27] Unusual charge on atom 42 number of radical electrons set to zero
ProtBert: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2181/2181 [00:00<00:00, 3879.48it/s]


  0%|          | 0/4491 [00:00<?, ?it/s]

Morgan:  59%|█████████████████████████████████████████████████████████████████████████████████                                                         | 2650/4510 [00:01<00:00, 2983.89it/s][15:06:09] Unusual charge on atom 0 number of radical electrons set to zero
[15:06:09] Unusual charge on atom 42 number of radical electrons set to zero
ProtBert: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2180/2180 [00:00<00:00, 3620.68it/s]


  0%|          | 0/4487 [00:00<?, ?it/s]

ProtBert: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1198/1198 [00:00<00:00, 3615.45it/s]


  0%|          | 0/6186 [00:00<?, ?it/s]

ProtBert: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 379/379 [00:00<00:00, 3918.81it/s]


  0%|          | 0/68 [00:00<?, ?it/s]

In [97]:
for dset in dset_list:
    display_results(*results[dset])
    print('----------------------')

Data Set biosnap:
Total drugs in training set: 4491
Train only tasks: 1529
Number of test tasks skipped: 91
Pct of test tasks skipped: 3.07%
Average per-task AUPR: 0.8567428675403678
Overall AUPR: 0.6412202615305818 +- 0.0
----------------------
Data Set biosnap_prot:
Total drugs in training set: 4487
Train only tasks: 1556
Number of test tasks skipped: 104
Pct of test tasks skipped: 3.55%
Average per-task AUPR: 0.8445605884426521
Overall AUPR: 0.6169060728797731 +- 0.0
----------------------
Data Set bindingdb:
Total drugs in training set: 6186
Train only tasks: 2865
Number of test tasks skipped: 2372
Pct of test tasks skipped: 71.42%
Average per-task AUPR: 0.8508353930972035
Overall AUPR: 0.5164448652661519 +- 0.0
----------------------
Data Set davis:
Total drugs in training set: 68
Train only tasks: 0
Number of test tasks skipped: 0
Pct of test tasks skipped: 0.00%
Average per-task AUPR: 0.4777958906068247
Overall AUPR: 0.31959984472067293 +- 0.0
----------------------
